In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive/IML PRoject

/content/drive/My Drive/IML PRoject


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

In [58]:
# %tensorflow_version 2.x

import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import PReLU
from tensorflow.keras.layers import MaxPool3D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import AveragePooling3D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import UpSampling3D
from tensorflow.keras.models import Model

In [31]:
data_path = r'C:\Users\kalya\OneDrive - University of Illinois at Chicago\!UIC\!Semesters\2nd Sem\Courses\CS 412 IML\Project\Alzheimers\Implementations\Our Implementations\!Pure Deep Learning\2\tf_datasets'
batch_size = 32

In [32]:
def decode(serialized_example):
    """
    Parses an image and label from the given `serialized_example`.
    It is used as a map function for `dataset.map`
    """

    features = tf.compat.v1.parse_single_example(
        serialized_example,
        features={
            'img_channels': tf.compat.v1.FixedLenFeature([], tf.int64),
            'img_height': tf.compat.v1.FixedLenFeature([], tf.int64),
            'img_width': tf.compat.v1.FixedLenFeature([], tf.int64),
            'img_raw': tf.compat.v1.FixedLenFeature([], tf.string),
            'sex': tf.compat.v1.FixedLenFeature([], tf.string),
            'age': tf.compat.v1.FixedLenFeature([], tf.float32),
            'label': tf.compat.v1.FixedLenFeature([], tf.string),
        })
    
    
    
    height = tf.cast(features['img_height'], tf.int32)
    width = tf.cast(features['img_width'], tf.int32)
    channels = tf.cast(features['img_channels'], tf.int32)
    image = tf.compat.v1.decode_raw(features['img_raw'], tf.float64)
    image = tf.reshape(image, (channels, height, width, 1))
    label = tf.compat.v1.decode_raw(features['label'], tf.float64)
    sex = tf.cast(features['sex'], tf.string)
    age = tf.cast(features['age'], tf.float32)
    return image, label

In [8]:
train_dataset = tf.data.TFRecordDataset(os.path.join(data_path, 'train.tfrecords'), compression_type='GZIP').map(decode).batch(batch_size)
val_dataset = tf.data.TFRecordDataset(os.path.join(data_path, 'validation.tfrecords'), compression_type='GZIP').map(decode).batch(batch_size)

In [ ]:
# class VariationalConvNetAutoEncoder(tf.keras.Model):
#     def __init__(self):
#         super(VariationalConvNetAutoEncoder, self).__init__()
#         self.input_3d = (62, 96, 96, 1)
#         self.pool_3d = (5, 5, 5)

#         self.model = Sequential()
        
#         self.model.add(tf.keras.layers.InputLayer(input_shape=self.input_3d))
#         self.conv1 = Conv3D(16, 3, activation='relu', padding='same')
#         self.maxp1 = MaxPool3D(self.pool_3d, padding='same')
#         self.conv2 = Conv3D(8, 3, activation='relu', padding='same')
#         self.maxp2 = MaxPool3D(self.pool_3d, padding='same')
#         self.conv3 = Conv3D(8, 3, activation='relu', padding='same')
        
#         # self.mu = MaxPool3D(self.pool_3d, padding='same')
#         # self.sigma = MaxPool3D(self.pool_3d, padding='same')
        
#         self.conv4 = Conv3D(8, 3, activation='relu', padding='same')
#         self.upsample1 = UpSampling3D(self.pool_3d)
#         self.conv5 = Conv3D(8, 3, activation='relu', padding='same')
#         self.upsample2 = UpSampling3D(self.pool_3d)
#         self.conv6 = Conv3D(16, 3, activation='relu')
#         self.upsample3 = UpSampling3D(self.pool_3d)
#         self.conv7 = Conv3D(1, 3, activation='sigmoid', padding='same')
        
#     def encoder(self, x):
#         x = self.conv1(x)
#         x = self.maxp1(x)
#         x = self.conv2(x)
#         x = self.maxp2(x)
#         x = self.conv3(x)
#         mu = self.mu(x)
#         sigma = self.sigma(x)
#         return mu, sigma
    
#     def decoder(self, x):
#         x = self.conv4(x)
#         x = self.upsample1(x)
#         x = self.conv5(x)
#         x = self.upsample2(x)
#         x = self.conv6(x)
#         x = self.upsample3(x)
#         x = self.conv7(x)
#         return x
    
#     def sample_from_mu_sigma(self, mu, sigma):
#         std = tf.exp(0.5 * sigma)
#         eps = tf.random_normal(shape=std.shape)
#         return mu + eps * std
        
#     def call(self, x):
#         mu, sigma = self.encoder(x)
#         z = self.sample_from_mu_sigma(mu, sigma)
#         y = self.decoder(z)
#         return y, mu, sigma
        
# def loss(x, x_bar, mu, sigma):
#   return tf.losses.mean_squared_error(x, x_bar) - (0.5 * tf.reduce_sum(1 + sigma - tf.math.pow(mu, 2) - tf.exp(sigma)))
# def grad(model, inputs, targets):
#   with tf.GradientTape() as tape:
#     reconstruction, mu, sigma = model(inputs)
#     loss_value = loss(targets, reconstruction, mu, sigma)
#   return loss_value, tape.gradient(loss_value, model.trainable_variables), reconstruction, mu, sigma




In [59]:
# def encoder(model):
#     model.add(Conv3D(filters=16,
#                      kernel_size=2,
#                      kernel_regularizer=kernel_regularizer,
#                      kernel_initializer=he_normal))
#     model.add(LeakyReLU())
#     model.add(BatchNormalization())
#     model.add(MaxPool3D(pool_size=pool_3d, name='pool1'))

#     model.add(Conv3D(filters=8,
#                      kernel_size=2,
#                      kernel_regularizer=kernel_regularizer,
#                      kernel_initializer=he_normal))
#     model.add(LeakyReLU())
#     model.add(BatchNormalization())
#     model.add(MaxPool3D(pool_size=pool_3d, name='pool2'))

#     model.add(Conv3D(filters=8,
#                      kernel_size=2,
#                      kernel_regularizer=kernel_regularizer,
#                      kernel_initializer=he_normal))
#     model.add(LeakyReLU())
#     model.add(BatchNormalization())
#     model.add(MaxPool3D(pool_size=pool_3d, name='pool3'))
#     return model

# def decoder(model):
#     model.add(Conv3D(filters=8,
#                      kernel_size=2,
#                      kernel_regularizer=kernel_regularizer,
#                      kernel_initializer=he_normal))
#     model.add(LeakyReLU())
#     model.add(BatchNormalization())
#     model.add(UpSampling3D(pool_3d, name='Up1'))

#     model.add(Conv3D(filters=16,
#                      kernel_size=2,
#                      kernel_regularizer=kernel_regularizer,
#                      kernel_initializer=he_normal))
#     model.add(LeakyReLU())
#     model.add(BatchNormalization())
#     model.add(UpSampling3D(pool_3d, name='Up2'))

#     model.add(Conv3D(filters=1,
#                      kernel_size=2,
#                      kernel_regularizer=kernel_regularizer,
#                      kernel_initializer=he_normal, activation='sigmoid'))
#     model.add(BatchNormalization())
#     return model

In [66]:
labels = pd.read_csv(r'C:\Users\kalya\OneDrive - University of Illinois at Chicago\!UIC\!Semesters\2nd Sem\Courses\CS 412 IML\Project\Alzheimers\Implementations\Our Implementations\!Pure Deep Learning\2\demographics.csv')

In [67]:
def load_datasets(type: str):
    if type not in ['train', 'test', 'valid']: raise Exception('Unsupported dataset type')
    train_valid_test = 0 if type == 'train' else 1 if type == 'valid' else 2
    i = 1
    data_path = r'C:\Users\kalya\OneDrive - University of Illinois at Chicago\!UIC\!Semesters\2nd Sem\Courses\CS 412 IML\Project\Alzheimers\Implementations\Our Implementations\!Pure Deep Learning\2\numpy_data'
    dataset = np.load(os.path.join(data_path, f'img_array_{type}_6k_{i}.npy'))
    while True:
        try:
            i += 1
            dataset = np.vstack((dataset, np.load(os.path.join(data_path, f'img_array_{type}_6k_{i}.npy'))))
        except FileNotFoundError:
            print(f'Loaded all {type} datasets')
            break
    # dataset = np.expand_dims(dataset, axis=1)
    for n in range(dataset.shape[0]):
        dataset[n, :, :] = dataset[n, :, :] / np.amax(dataset[n, :, :].flatten())
    print(f'Normalized {n+1} images')
    dataset = np.reshape(dataset, (-1, 62, 96, 96, 1))
    return dataset, labels[labels.train_valid_test == train_valid_test]

In [68]:
valid_data, valid_demo = load_datasets('valid')

Loaded all valid datasets
Normalized 26970 images


In [71]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [72]:
filters=[32, 64]
latentDim=16
chanDim = -1
tf.keras.backend.set_floatx('float64')
# define the input to the encoder
inputs = valid_data
x = inputs

# loop over the number of filters
for f in filters:
    # apply a CONV => RELU => BN operation
    x = Conv3D(f, 3, strides=2, padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization(axis=chanDim)(x)

# flatten the network and then construct our latent vector
volumeSize = K.int_shape(x)
x = Flatten()(x)
latent = Dense(latentDim)(x)

# build the encoder model
encoder = Model(inputs, latent, name="encoder")

# start building the decoder model which will accept the
# output of the encoder as its inputs
latentInputs = Input(shape=(latentDim,))
x = Dense(np.prod(volumeSize[1:]))(latentInputs)
x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)

# loop over our number of filters again, but this time in
# reverse order
for f in filters[::-1]:
    # apply a CONV_TRANSPOSE => RELU => BN operation
    x = Conv3DTranspose(f, 3, strides=2, padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization(axis=chanDim)(x)

# apply a single CONV_TRANSPOSE layer used to recover the
# original depth of the image
x = Conv3DTranspose(1, 3, padding="same")(x)
outputs = Activation("sigmoid")(x)

# build the decoder model
decoder = Model(latentInputs, outputs, name="decoder")

# our autoencoder is the encoder + decoder
autoencoder = Model(inputs, decoder(encoder(inputs)),
    name="autoencoder")


ResourceExhaustedError: OOM when allocating tensor with shape[435,31,48,48,32] and type double on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:AddV2] name: batch_normalization_49/batchnorm/add/

In [57]:

kernel_regularizer = tf.keras.regularizers.l2(0.001)
he_normal = tf.keras.initializers.he_normal(seed=0)
lecun_normal = tf.keras.initializers.lecun_normal(seed=0)

input_3d = (62, 96, 96, 1)
pool_3d = (2, 2, 2)

from tensorflow.keras.models import Model as model

model.add(tf.keras.layers.InputLayer(input_shape=input_3d))

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-5),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
encoded_data = encoder(model).fit(train_dataset)


AttributeError: type object 'Model' has no attribute 'add'

In [46]:
# def create_model():
#     kernel_regularizer = tf.keras.regularizers.l2(0.001)
#     he_normal = tf.keras.initializers.he_normal(seed=0)
#     lecun_normal = tf.keras.initializers.lecun_normal(seed=0)

#     input_3d = (62, 96, 96, 1)
#     pool_3d = (5, 5, 5)

#     model = Sequential()
    
#     model.add(tf.keras.layers.InputLayer(input_shape=input_3d))
#     model.add(Conv3D(filters=8,
#                      kernel_size=5,
#                      kernel_regularizer=kernel_regularizer,
#                      kernel_initializer=he_normal))
#     model.add(BatchNormalization())
#     model.add(LeakyReLU())
#     model.add(MaxPool3D(pool_size=pool_3d, name='pool1'))

#     model.add(Conv3D(filters=8,
#                      kernel_size=5,
#                      kernel_regularizer=kernel_regularizer,
#                      kernel_initializer=he_normal))
#     model.add(BatchNormalization())
#     model.add(LeakyReLU())
#     model.add(MaxPool3D(pool_size=pool_3d, name='pool2'))

#     # model.add(Conv3D(filters=8,
#     #                  kernel_size=2,
#     #                  kernel_regularizer=kernel_regularizer,
#     #                  kernel_initializer=he_normal))
#     # model.add(BatchNormalization())
#     # model.add(LeakyReLU())
#     # model.add(MaxPool3D(pool_size=pool_3d, name='pool3'))

#     # model.add(Conv3D(filters=8,
#     #                  kernel_size=5,
#     #                  kernel_regularizer=kernel_regularizer,
#     #                  kernel_initializer=he_normal))
#     # model.add(BatchNormalization())
#     # model.add(LeakyReLU())
#     # model.add(MaxPool3D(pool_size=pool_3d, name='pool4'))

#     model.add(Flatten())
#     model.add(Dense(1024, name='dense1', kernel_regularizer=kernel_regularizer, kernel_initializer=he_normal))
#     model.add(LeakyReLU())
#     model.add(Dropout(0.5, name='dropout1'))

#     model.add(Dense(512, activation='relu', name='dense2', kernel_regularizer=kernel_regularizer, kernel_initializer=he_normal))
#     model.add(LeakyReLU())
#     model.add(Dropout(0.5, name='dropout2'))

#     model.add(Dense(3, activation='softmax', name='softmax', kernel_initializer=lecun_normal))

#     model.compile(loss='categorical_crossentropy',
#                   optimizer=tf.keras.optimizers.Adam(1e-5),
#                   metrics=[tf.keras.metrics.CategoricalAccuracy()])

#     return model

In [47]:
reducelr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy',
                                                         factor=0.2,
                                                         patience=5,
                                                         min_delta=0.01,
                                                         verbose=1)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(os.path.join('checkpoints', 'cp.ckpt'),
                                                         monitor='val_categorical_accuracy',
                                                         verbose=1,
                                                         save_best_only=True,
                                                         mode='max')
weights_callback = tf.keras.callbacks.ModelCheckpoint(os.path.join('weights', 'best_weights.h5'),
                                                      monitor='val_categorical_accuracy',
                                                      verbose=1,
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      mode='max')

callbacks_list = [weights_callback, checkpoint_callback, reducelr_callback]

In [48]:
model = create_AE()
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_35 (Conv3D)           (None, 61, 95, 95, 16)    144       
_________________________________________________________________
batch_normalization_32 (Batc (None, 61, 95, 95, 16)    64        
_________________________________________________________________
leaky_re_lu_29 (LeakyReLU)   (None, 61, 95, 95, 16)    0         
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 47, 47, 16)    0         
_________________________________________________________________
conv3d_36 (Conv3D)           (None, 29, 46, 46, 8)     1032      
_________________________________________________________________
batch_normalization_33 (Batc (None, 29, 46, 46, 8)     32        
_________________________________________________________________
leaky_re_lu_30 (LeakyReLU)   (None, 29, 46, 46, 8)    

In [ ]:
if os.path.exists(os.path.join('checkpoints', 'cp.ckpt')):
    model = load_model(os.path.join('checkpoints', 'cp.ckpt'))

In [ ]:
# 8.0000e-08

In [51]:
history = model.fit(train_dataset, validation_data=train_dataset, epochs=40, callbacks=callbacks_list, verbose=1)

Epoch 1/40
      1/Unknown - 0s 22ms/stepWARNING:tensorflow:Can save best model only with val_categorical_accuracy available, skipping.
      1/Unknown - 0s 25ms/step

DataLossError:  inflate() failed with error -3: incorrect header check
	 [[node IteratorGetNext (defined at <ipython-input-49-0e7deba41ea6>:1) ]] [Op:__inference_distributed_function_8647]

Function call stack:
distributed_function


In [ ]:
test_dataset = tf.data.TFRecordDataset(os.path.join(data_path, 'test.tfrecords'), compression_type='GZIP').map(decode).batch(batch_size)
results = model.evaluate(test_dataset)

15/15 [==============================] - 14s 902ms/step - loss: 4.0485 - categorical_accuracy: 0.4819


In [ ]:
t = []
for image, label in test_dataset:
  t.append(label)

In [ ]:
len(t)

15

In [ ]:
def df(t):
  y =[]
  for i in range(len(t)):
    for j in range(len(t[i])):
          max = np.argmax(t[i][j])
          y.append(max)
  return y



In [ ]:
y_test = df(t)
y_test = np.asarray(y_test)

In [ ]:
predictions = model.predict(test_dataset)

In [ ]:
predictions

array([[0.00653354, 0.11199015, 0.8814763 ],
       [0.9644184 , 0.02377447, 0.01180711],
       [0.00872518, 0.014667  , 0.9766078 ],
       ...,
       [0.31045234, 0.35707214, 0.3324755 ],
       [0.19253181, 0.7281903 , 0.07927796],
       [0.06457602, 0.07411502, 0.86130893]], dtype=float32)

In [ ]:
def cf(x):
    x = np.asarray(x)
    y =[]
    for i in range(len(x)):
        max = np.argmax(x[i])
        y.append(max)
    return np.asarray(y)
    

In [ ]:
# y_test =np.asarray([i for i in list(test_labels)])
# y_test

In [ ]:
y_pred = cf(predictions)

In [ ]:
conf = metrics.confusion_matrix(y_test, y_pred)
conf

array([[152,   3,   4],
       [ 23, 100,  34],
       [  7,  26, 120]])

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_pred)
f1_score = metrics.f1_score(y_test, y_pred, average = None)
recall = metrics.recall_score(y_test, y_pred, average= None)
print("The accuracy is " + str(accuracy))
print("F1 Scores are " + str(f1_score))
print("Recall Scores are " + str(recall))

The accuracy is 0.7931769722814499
F1 Scores are [0.8914956  0.6993007  0.77170418]
Recall Scores are [0.95597484 0.63694268 0.78431373]
